<a href="https://colab.research.google.com/github/Krati012/IDP-2020/blob/master/TRRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
!ls "/content/gdrive/My Drive/CTW_2019"

 AE_Model_Conv2.h5		    CTW2019_mean_sqr2.h5
 AE_Model_Conv.h5		    CTW2019_mean_sqr.h5
 AutoEncoderModel1.h5		    data2019.h5
 AutoEncoderModel2.h5		    data_feat.h5
 AutoEncoderModel.h5		    ensemble_NDR_CNN_SNR_finetune.h5
 best_AE1.h5			    ensemble_NDR_CNN_SNR.h5
 best_AE2.h5			    h_Estimated_CTW_Train.h5
 best_AE3.h5			    Histogram_MSE.png
 best_AE4.h5			    Histogram.png
 best_cnn.h5			    Histogram_RMSE.png
 best_cnn_ndr2.h5		    NDR.ipynb
 best_cnn_ndr3.h5		   'PCA(1056)+CNN'
 best_cnn_ndr4.h5		   'PCA(66)+CNN'
 best_cnn_ndr5_adam.h5		    PCA+MLP_Hist.png
 best_cnn_ndr.h5		    PCA+MLP.ipynb
 best.h5			    PCA_Unsupervised.sav
 Best_PCA_CNN_2.h5		    r_Position_CTW_Train.h5
 Best_PCA_CNN.h5		    saved_model.pb
 best_PCA.h5			    SimpleAE.h5
'Copy of AE.ipynb'		    SimpleAE_pred.h5
'Copy of PCA+CNN'		    SNR_CTW_Train.h5
 CTW2019_Dataset_h5.zip		    test.out
 CTW2019_logarithimic_mean_sqr.h5


In [5]:
import numpy as np # linear algebra
import keras
import os
import h5py
import pandas as pd
import time
import matplotlib.pyplot as plt
import tensorflow as tf

In [6]:
'''#getting and processing data
#h_Estimated   ---> shape = (2, 924, 16, 17486)
#r_postition ---> shape = (3, 17486)
#SNR_Est    ---> shape = (16, 17486)


batch_size = 30
epochs = 50
data_augmentation = False
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cnn_training.h5'
'''

"#getting and processing data\n#h_Estimated   ---> shape = (2, 924, 16, 17486)\n#r_postition ---> shape = (3, 17486)\n#SNR_Est    ---> shape = (16, 17486)\n\n\nbatch_size = 30\nepochs = 50\ndata_augmentation = False\nsave_dir = os.path.join(os.getcwd(), 'saved_models')\nmodel_name = 'keras_cnn_training.h5'\n"

In [7]:
f = h5py.File('/content/gdrive/My Drive/CTW_2019/h_Estimated_CTW_Train.h5','r')
H = f['h_Estimated']
f = h5py.File('/content/gdrive/My Drive/CTW_2019/r_Position_CTW_Train.h5','r')
pos = f['r_Position']


In [8]:
print(H.shape)
H = np.array(H)
pos = np.array(pos)

(2, 924, 16, 17486)


In [9]:
data = H[:,:,:,:1000]
print(data.shape)
xyz = pos[:,:1000]
print(xyz.shape)

(2, 924, 16, 1000)
(3, 1000)


In [13]:
H = data[:,:,:,:900]
H = H.swapaxes(0, -1)
pos = xyz[:,:900]

H_test = data[:,:,:,900:]
H_test = H_test.swapaxes(0, -1)
pos_test = xyz[:,900:]

print(H.shape)
print(pos_test.shape)

(900, 924, 16, 2)
(3, 100)


In [14]:
'''Both Single CFR'''
H1 = H[1]
H2 = H_test[1]

print(H2.shape)

Yd1 = np.sum(((H1*H1)[:,:,0]) + ((H1*H1)[:,:,1]), axis=0)
Yd2 = np.sum((H2*H2)[:,:,0] + (H2*H2)[:,:,1] , axis=0)
print(Yd2.shape)   #We have 16 links

H11 = H1[:,:,0] + H1[:,:,1] * 1j
H22 = H2[:,:,0] - H2[:,:,1] * 1j
G = H11 * H22
print(G.shape)


Nser = 1024
Nu = 924
G = np.concatenate((G, np.zeros((Nser-Nu, 16))), axis=0)

g = np.fft.fftn(G, axes=(0,))
print(g.shape)

phi_d = np.ndarray.max((np.square(np.abs(g))), axis=0)
phi_d = np.divide(phi_d,np.multiply(Yd1, Yd2))
print(phi_d.shape)

w_d = np.sqrt(np.multiply(Yd1,Yd2))/(np.sqrt(np.sum(Yd1)) * np.sum(Yd2))
print(w_d.shape)

TRRS = np.sum(np.multiply(w_d,np.sqrt(phi_d)))
TRRS = TRRS**2

print(TRRS)

(924, 16, 2)
(16,)
(924, 16)
(1024, 16)
(16,)
(16,)
0.02238512572668457


In [21]:
'''H1 grid, H2 single CFR'''
X = H

def trrs(H1, H2):
  Yd1 = np.sum((H1*H1)[:,:,:,0] + (H1*H1)[:,:,:,1] , axis=1)
  Yd2 = np.sum((H2*H2)[:,:,0] + (H2*H2)[:,:,1] , axis=0)
  #print((Yd1*Yd2).shape)

  H11 = H1[:,:,:,0] + H1[:,:,:,1] * 1j
  H22 = H2[:,:,0] - H2[:,:,1] * 1j
  G = H11 * H22
  
  Nser = 1024
  Nu = 924
  G = np.concatenate((G, np.zeros((900,Nser-Nu, 16))), axis=1)
  #print(G.shape)

  g = np.fft.fftn(G, axes=(1,))
  #print(g.shape)

  phi_d = np.ndarray.max((np.square(np.abs(g))), axis=1)
  phi_d = np.divide(phi_d,(Yd1*Yd2))
  #print(phi_d.shape)

  w_d = np.sqrt((Yd1 * Yd2))
  w_d = np.transpose(w_d)/(np.sqrt(np.sum(Yd1, axis=1)) * np.sum(Yd2))
  w_d = np.transpose(w_d)
  #print(w_d.shape)
  
  TRRS = np.sum(np.multiply(w_d,np.sqrt(phi_d)), axis=1)
  TRRS = np.square(TRRS)

  idx = np.argmax(TRRS)

  return pos[:,idx]


print(trrs(X,H2))
print(pos_test[:,1])
error = np.sqrt(((trrs(X,H2) - pos_test[:,1]) ** 2).mean())
print(error)
  

[ 3.4761  0.4693 -0.5203]
[ 3.4761  0.4693 -0.5203]
0.0


In [22]:
'''Complete train and test set'''
ans_grid = np.zeros((pos_test.shape))

for i in range(0,100):
  for j in range(0,3):
    ans_grid[j][i] = trrs(X,H_test[i])[j]

err = np.sqrt(((ans_grid - pos_test) ** 2).mean())

print(err)


0.8966715157365786
